# Projet NLP - Groupe 1 - Nohossat, Valérie, Williams

## Consignes

Vous venez d'ouvrir un hôtel. Comme vous n'êtes pas sûr de la qualité de votre établissement, vous permettez aux personnes de poster des commentaires mais pas de mettre de note. Cependant, vous voulez quand même déterminer si le commentaire est positif ou négatif.  

Pour cela, vous allez scrapper des commentaires sur booking et leur note associée afin de faire tourner un algorithme de classification pour faire des prédictions sur vos propres commentaires.

## Application

https://projetnlp.herokuapp.com/

## 1 - Récuperation des données

Site de référence : https://www.booking.com

### Contraintes du scraping

On a récupéré les commentaires des hôtels de plusieurs villes françaises telles que **Paris, Marseille, Lyon, etc...**  

Booking ne permet pas d'ouvrir la page d'un hôtel dans le même onglet donc on a procédé en 2 temps :

- Récupération des liens vers les hôtels avec des commentaires (on ignore ceux qui n'ont aucun commentaire)
- Récupération des commentaires pour chaque hôtel présélectionné

Cette démarche nous permet de reprendre le scraping en cas de crash puisque les liens vers les hôtels sont sauvegardés avant de passer à la deuxième étape.

<img width="1000px" src='booking.gif' alt='booking_website'>

### Données récupérées

Pour chaque hôtel, on récupère au maximum 220 commentaires et les informations suivantes:

- **NOM** : nom du client ayant laissé le commentaire
- **PAYS** : pays de provenance du client
- **FAVORITE** : est-ce que le client a marqué l'établissement parmi ces favoris ?
- **DATE** : date d'envoi du commentaire
- **TITRE** : titre du commentaire laissé par le client
- **BONS POINTS** : les aspects positifs de l'expérience
- **MAUVAIS POINTS** : les aspects négatifs de l'expérience
- **NOTE** : la note laissée par le client
- **TYPE ETABLISSEMENT**: le type de l'établissement (Appartement, Hôtel, etc..)
- **LIEU** : Ville de l'établissement
- **NOTE ETABLISSEMENT** : note moyenne laissée par l'ensemble des commentateurs

<img width="600px" src='commentaire_booking.png' alt='booking_commentaire'>

### Résilience du scraping

Plusieurs actions ont été mises en place pour rendre le scraping résilient: 

- relance de la page en cas d'apparition du pop-up 'Etes-vous humain?'
- timeout de 8min pour récupérer les 220 commentaires par hôtel
- multiprocessing (1 process par ville)
- backup après chaque commentaire récupéré


### Résultats

Récupération de 30 946 commentaires

## 2 - Analyse des données

In [18]:
import pandas as pd
df = pd.read_csv('booking.csv')

### Exploration

In [5]:
df.head(5)

,nom,pays,favorite,date,titre,bons_points,mauvais_points,note,type_etablissement,lieu,note_etablissement
0,Caroline,Belgique,1.0,2018-07-30 00:00:00,Exceptionnel,"Établissement recent, propre, soigne, chouette...",Rien,10,Hôtel,Paris,"8,5"
1,Olivier51,France,0.0,2020-03-16 00:00:00,Fabuleux,"Accueil très chaleureux, chambre calme...tout ...",None,"9,0",Hôtel,Paris,"8,5"
2,Marc,France,0.0,2020-03-11 00:00:00,Exceptionnel,Tout bié sauf le wifi catastrophiquement lent!...,None,"9,6",Hôtel,Paris,"8,5"
3,Marie-claire,France,0.0,2020-03-08 00:00:00,Exceptionnel,None,Hôtel parfait. Quartier sympa.,10,Hôtel,Paris,"8,5"
4,Pierre,France,0.0,2020-03-06 00:00:00,Fabuleux,Excellent rapport qualité prix. Personnel très...,None,"9,0",Hôtel,Paris,"8,5"


In [14]:
print(f'Le dataset a {df.shape[0]} lignes et {df.shape[1]} colonnes.')

Le dataset a 30946 lignes et 11 colonnes.


#### Data Types

In [20]:
df.dtypes

nom                    object
pays                   object
favorite              float64
date                   object
titre                  object
bons_points            object
mauvais_points         object
note                   object
type_etablissement     object
lieu                   object
note_etablissement     object
dtype: object

Les statistiques descriptives ne sont pas concluantes parce que les colonnes sont pour la plupart du type string.

### Objectif

Notre objectif est de savoir si un commentaire est positif ou négatif.   
Pour cela, on doit se baser sur toutes les colonnes qui regroupent une partie du commentaire soit les colonnes :
- **TITRE**, 
- **BONS POINTS**
- **MAUVAIS POINTS**.

On ne conservera que ces colonnes pour la suite de notre analyse.

### Nettoyage des valeurs manquantes

On décide de remplacer les valeurs nulles (‘None’, NaN) par des espaces vides.   
Un espace vide correspond donc à une absence de commentaire.

In [15]:
df.isnull().values.any()
#affichage du nombre de valeurs manquantes selon chaque variable
df.isnull().sum()

nom                   395
pays                  410
favorite               15
date                  410
titre                 408
bons_points           391
mauvais_points        304
note                  410
type_etablissement     15
lieu                   15
note_etablissement     15
dtype: int64

### Nettoyage des données

Le nettoyage des données est nécessaire avant de pouvoir poursuivre l'analyse des commentaires. On va réaliser les actions suivantes : 

- Mettre les mots en minuscules
- Supprimer la ponctuation
- Supprimer les caractères numériques
- Supprimer les mots dont la longueur est inférieure à 2
- Supprimer les mots vides (conjonctions)

### Fréquence des mots et nuage de mots

### Lemmatisation

TBD

### Correction orthographique

TBD

### Que faire des mots en anglais ?

TBD

### Calcul de la polarité et de la subjectivité

#### Qu'est-ce que la polarité ?

>le commentaire envoyé est-il positif, négatif ou neutre ? 

La polarité est ce que l'on cherche à déterminer.

#### Qu'est-ce que la subjectivité ?

est-ce que le texte exprime une information objective ou est-ce qu’il donne une opinion subjective.

**Pourquoi cherche-t-on a déterminer la subjectivité ?**

Notre dataset final aura donc 2 colonnes: 
- variable indépendante : commentaire (concaténation des colonnes titre, bons points et mauvais points)
- target : polarité

## 3 - Modélisation

4 approches

## 4 - Résultats et Evaluation

TBD

## 5 - Conclusions et Difficultés rencontrées

### Conclusions

TBD

#### Scraping

- **gérer les coupures internet et les code recaptcha** : il faut relancer le process automatiquement après coupure ou lorsqu'un popup 'Etes-vous humain' empêche toute interaction.
- **booking.com ouvre forcément les pages établissement sur un nouvel onglet**: cela ralentit le code puisqu'il faut prendre en compte le temps de latence et les codes intempésitfs de booking (cookie / recaptcha)
- **gérer le multiprocessing** : pour récupérer le plus de données possibles, on a utilisé le multiprocessing. Mais on s'est rendu compte que la gestion des exceptions est ici primordiale parce que malgré les exceptions, le code doit continuer de scraper le site. Ce qui compte c'est d'en scraper le plus possible et non de pouvoir scrapper tous les commentaires existants. 


### Analyse des données

TBD

### Modélisation

TBD

### Création de l'application

TBD